In [1]:
from numpy.random import seed
seed(1)

import numpy as np
import pandas as pd

In [2]:
import tensorflow
tensorflow.keras.__version__

'2.2.4-tf'

In [3]:
data = pd.read_csv('../datasets/spotify_data.csv')
data.head()

,track,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target,decade
0,Lucky Man,Montgomery Gentry,spotify:track:4GiXBCUF7H6YfNQsnBRIzl,0.578,0.471,4,-7.270,1,0.0289,0.368000,0.00000,0.159,0.532,133.061,196707,4,30.88059,13,1,2000
1,On The Hotline,Pretty Ricky,spotify:track:1zyqZONW985Cs4osz9wlsu,0.704,0.854,10,-5.477,0,0.1830,0.018500,0.00000,0.148,0.688,92.988,242587,4,41.51106,10,1,2000
2,Clouds Of Dementia,Candlemass,spotify:track:6cHZf7RbxXCKwEkgAZT4mY,0.162,0.836,9,-3.009,1,0.0473,0.000111,0.00457,0.174,0.300,86.964,338893,4,65.32887,13,0,2000
3,"Heavy Metal, Raise Hell!",Zwartketterij,spotify:track:2IjBPp2vMeX7LggzRN3iSX,0.188,0.994,4,-3.745,1,0.1660,0.000007,0.07840,0.192,0.333,148.440,255667,4,58.59528,9,0,2000
4,I Got A Feelin',Billy Currington,spotify:track:1tF370eYXUcWwkIvaq3IGz,0.630,0.764,2,-4.353,1,0.0275,0.363000,0.00000,0.125,0.631,112.098,193760,4,22.62384,10,1,2000


In [5]:
data_10s = data[data["decade"] == 2010]
data_10s.head()

,track,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target,decade
5872,Wild Things,Alessia Cara,spotify:track:2ZyuwVvV6Z3XJaXIFbspeE,0.741,0.626,1,-4.826,0,0.0886,0.02000,0.000,0.0828,0.706,108.029,188493,4,41.18681,10,1,2010
5873,Surfboard,Esquivel!,spotify:track:61APOtq25SCMuK0V5w2Kgp,0.447,0.247,5,-14.661,0,0.0346,0.87100,0.814,0.0946,0.250,155.489,176880,3,33.18083,9,0,2010
5874,Love Someone,Lukas Graham,spotify:track:2JqnpexlO9dmvjUMCaLCLJ,0.550,0.415,9,-6.557,0,0.0520,0.16100,0.000,0.1080,0.274,172.065,205463,4,44.89147,9,1,2010
5875,Music To My Ears (feat. Tory Lanez),Keys N Krates,spotify:track:0cjfLhk8WJ3etPTCseKXtk,0.502,0.648,0,-5.698,0,0.0527,0.00513,0.000,0.2040,0.291,91.837,193043,4,29.52521,7,0,2010
5876,Juju On That Beat (TZ Anthem),Zay Hilfigerrr & Zayion McCall,spotify:track:1lItf5ZXJc1by9SbPeljFd,0.807,0.887,1,-3.892,1,0.2750,0.00381,0.000,0.3910,0.780,160.517,144244,4,24.99199,8,1,2010


In [4]:
X = data.drop(['track', 'artist', 'uri', 'target', 'decade', 'chorus_hit', 'sections'], axis=1)
y = data["target"]
print(X.shape, y.shape)

(41106, 13) (41106,)


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [7]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [23]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=30, activation='relu', input_dim=13))
model.add(Dense(units=15, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [24]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 30)                420       
_________________________________________________________________
dense_7 (Dense)              (None, 15)                465       
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 32        
Total params: 917
Trainable params: 917
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

Train on 30829 samples
Epoch 1/100
30829/30829 - 4s - loss: 0.5540 - accuracy: 0.7096
Epoch 2/100
30829/30829 - 3s - loss: 0.5129 - accuracy: 0.7436
Epoch 3/100
30829/30829 - 3s - loss: 0.5055 - accuracy: 0.7480
Epoch 4/100
30829/30829 - 3s - loss: 0.5023 - accuracy: 0.7485
Epoch 5/100
30829/30829 - 3s - loss: 0.4995 - accuracy: 0.7519
Epoch 6/100
30829/30829 - 3s - loss: 0.4975 - accuracy: 0.7537
Epoch 7/100
30829/30829 - 3s - loss: 0.4957 - accuracy: 0.7559
Epoch 8/100
30829/30829 - 3s - loss: 0.4950 - accuracy: 0.7563
Epoch 9/100
30829/30829 - 3s - loss: 0.4932 - accuracy: 0.7575
Epoch 10/100
30829/30829 - 3s - loss: 0.4919 - accuracy: 0.7589
Epoch 11/100
30829/30829 - 3s - loss: 0.4913 - accuracy: 0.7594
Epoch 12/100
30829/30829 - 3s - loss: 0.4911 - accuracy: 0.7591
Epoch 13/100
30829/30829 - 4s - loss: 0.4895 - accuracy: 0.7608
Epoch 14/100
30829/30829 - 3s - loss: 0.4890 - accuracy: 0.7606
Epoch 15/100
30829/30829 - 3s - loss: 0.4883 - accuracy: 0.7614
Epoch 16/100
30829/30829 -

In [27]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

10277/10277 - 1s - loss: 0.4713 - accuracy: 0.7717
Normal Neural Network - Loss: 0.47126528998867506, Accuracy: 0.7717232704162598


In [28]:
encoded_predictions = model.predict_classes(X_test_scaled[:10])

In [29]:
print(f"Predicted models: {encoded_predictions}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted models: [0 1 1 1 1 0 1 1 1 0]
Actual Labels: [0, 0, 0, 0, 1, 0, 1, 0, 1, 1]


In [30]:
correlation=data.corr().unstack().sort_values(ascending=False).drop_duplicates()
correlation[:10]

decade        decade          1.000000
duration_ms   sections        0.888952
energy        loudness        0.772628
danceability  valence         0.553858
target        danceability    0.346097
energy        valence         0.341427
loudness      decade          0.329926
energy        decade          0.324340
loudness      target          0.286034
danceability  loudness        0.274160
dtype: float64

In [31]:
model.save("model_trained_2010s_13_feat.h5")